In [37]:
import pretty_midi
import pandas as pd
import json
import glob
import os

In [27]:
master_dict = []

In [22]:
master_dict

[]

In [4]:
def extract_guitar_and_drums(midi_file):
    """This function extracts the guitar and drum tracks from a midi file.
       The input is a path to a midi file (for example: 'raw_data/song_name.mid') in string format
       The output is a dictionary with the song name, guitar track and drum track"""    
    
    mid = pretty_midi.PrettyMIDI(midi_file)
    
    guitars = []
    lengths_guitar = []
    drums = []
    lengths_drums = []
    
    for instrument in mid.instruments:
        if instrument.is_drum:
            drums.append(instrument)
            lengths_drums.append(len(instrument.notes))
        if (instrument.program >= 25) and (instrument.program <= 31):
            guitars.append(instrument)
            lengths_guitar.append(len(instrument.notes))
    drum_track = drums[lengths_drums.index(max(lengths_drums))]
    guitar_track = guitars[lengths_guitar.index(max(lengths_guitar))]
    song_title = os.path.splitext(os.path.basename(midi_file))[0]
    
    song_dict = {'title': song_title,
                 'down_beats': mid.get_downbeats(),
                 'guitar': guitar_track,
                 'drums': drum_track
                }
    
    return song_dict

In [28]:
def tracks_to_bars(song_dict: dict) -> dict:
    """
    This function accepts a dictionary as an input with 4 keys: 'title', 'down_beats', 'guitar', 'drums'.
    The function takes the guitar and drums, both pretty_midi instrument objects, and cuts them up into a sequence of individual bars.
    The output is a dictionary that contains the following keys/values: song_title, a list of guitar bars, a list of drum bars, and a list of the song's downbeats
    """
    
    new_dict={}
    new_dict['song_title']=song_dict['title']
    guitar = song_dict['guitar']
    drums = song_dict['drums']
    down_beats_array = song_dict['down_beats']


    guitar_bars_list = []
    drums_bars_list = []

    for index, start_time in enumerate(down_beats_array):
        if index < len(down_beats_array) - 1:
            end_time = down_beats_array[index+1] 
        else:
            end_time = down_beats_array[index] + 2
            
        guitar_bar = []
        drums_bar = []
        for guitar_note in guitar.notes:
            if (guitar_note.start >= start_time) and (guitar_note.end < end_time):
                guitar_bar.append(guitar_note)    

        for drum_note in drums.notes:
            if (drum_note.start >= start_time) and (drum_note.end < end_time):
                drums_bar.append(drum_note)
                  
        drums_bars_list.append(drums_bar)
        guitar_bars_list.append(guitar_bar)

    new_dict['guitar_bars'] = guitar_bars_list
    new_dict['drum_bars'] = drums_bars_list
    new_dict['down_beats'] = down_beats_array.tolist()
    return new_dict
    